#General Imports

In [ ]:
import os, inspect, sys
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16,8

In [ ]:
script_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.join(script_dir, os.pardir, os.pardir))

In [ ]:
from opengrid.library import config
c = config.Config()

In [ ]:
sys.path.append(c.get('tmpo', 'folder'))

try:
    if os.path.exists(c.get('tmpo', 'data')):
        path_to_tmpo_data = c.get('tmpo', 'data')
except:
    path_to_tmpo_data = None

#Import Houseprint Modules

In [ ]:
from opengrid.library.houseprint import houseprint

#Test

##Houseprint

In [ ]:
hp = houseprint.Houseprint(gjson = c.get('houseprint','json'))

In [ ]:
hp

In [ ]:
hp.sites

In [ ]:
hp.get_devices()

In [ ]:
hp.get_sensors('water')

A Houseprint object can be saved as a pickle. It loses its tmpo session however (connections cannot be pickled)

In [ ]:
hp.save('new_houseprint.pkl')

In [ ]:
hp = houseprint.load_houseprint_from_file('new_houseprint.pkl')

###TMPO

The houseprint, sites, devices and sensors all have a get_data method. In order to get these working for the fluksosensors, the houseprint needs a tmpo session

In [ ]:
hp.init_tmpo(path_to_tmpo_data)

In [ ]:
hp.sync_tmpos()

###Lookup sites, devices, sensors

In [ ]:
hp.find_site(1)

In [ ]:
hp.find_device('FL03001552')

In [ ]:
hp.find_sensor('53b1eb0479c83dee927fff10b0cb0fe6')

###Get Data

In [ ]:
head = pd.Timestamp('20150617')
tail = pd.Timestamp('20150618')
hp.get_data(sensortype='water', head=head,tail=tail).plot()

##Site

In [ ]:
site = hp.find_site(1)
site

In [ ]:
print site.size
print site.inhabitants
print site.postcode
print site.construction_year
print site.k_level
print site.e_level
print site.epc_cert

In [ ]:
site.devices

In [ ]:
site.get_sensors('electricity')

In [ ]:
head = pd.Timestamp('20150617')
tail = pd.Timestamp('20150618')
site.get_data(sensortype='electricity', head=head,tail=tail).plot()

##Device

In [ ]:
device = hp.find_device('FL03001552')
device

In [ ]:
device.key

In [ ]:
device.get_sensors('gas')

In [ ]:
head = pd.Timestamp('20150617')
tail = pd.Timestamp('20150618')
device.get_data(sensortype='electricity', head=head,tail=tail).plot()

##Sensor

In [ ]:
sensor = hp.find_sensor('53b1eb0479c83dee927fff10b0cb0fe6')
sensor

In [ ]:
sensor.key

In [ ]:
sensor.type

In [ ]:
sensor.description

In [ ]:
sensor.system

In [ ]:
sensor.unit

In [ ]:
head = pd.Timestamp('20150617')
tail = pd.Timestamp('20150618')
sensor.get_data(head,tail).plot()